# Long Lab 3 – COVID-19 Tweet Analysis  
**Course:** Data Mining – Fall 2025  
**University:** University of Rochester  
**Student:** Mouhamed Mbengue  



## Part I: Setting Up the Data
### Q1: Import and Clean Tweets Dataset

In this section, we:
- Import the original tweets dataset.  
- Drop duplicate entries based on `status_id`.  
- Randomly sample 500,000 tweets (if available).  
- Import the U.S. states reference file.  
- Filter tweets that reference exactly one U.S. state.  
- Report the number of rows before and after cleaning, percentage of data lost, and elapsed time.


In [ ]:
import time
import pandas as pd
import numpy as np
import string
import string
import nltk
from nltk.corpus import stopwords

# nltk.download('stopwords')  # Uncomment if running for the first time

print("Libraries imported successfully.")


Libraries imported successfully.


In [1]:
import pandas as pd, time

start = time.time()
sample = pd.read_csv("us_tweets_final.csv", dtype=str, nrows=5000)
print("Loaded sample shape:", sample.shape)
print("Columns:", list(sample.columns))
print("Time:", round(time.time() - start, 2), "sec")



Loaded sample shape: (5000, 26)
Columns: ['Unnamed: 0', 'user_id', 'status_id', 'created_at', 'screen_name', 'text', 'reply_to_screen_name', 'is_quote', 'is_retweet', 'favorite_count', 'retweet_count', 'quote_count', 'reply_count', 'hashtags', 'name', 'location', 'description', 'url', 'protected', 'followers_count', 'friends_count', 'listed_count', 'statuses_count', 'favourites_count', 'account_created_at', 'state']
Time: 0.05 sec


In [4]:

import pandas as pd, numpy as np, time
start_time = time.time()

# Efficiently sample ~500,000 rows while reading (without loading entire 8M into memory)
row_count = sum(1 for _ in open("us_tweets_final.csv")) - 1  # subtract header
sample_size = 500000
skip = sorted(np.random.choice(np.arange(1, row_count + 1), row_count - sample_size, replace=False))

tweets_us_df = pd.read_csv(
    "us_tweets_final.csv",
    dtype=str,
    encoding="utf-8",
    skiprows=skip,
    low_memory=False
)

print("Sampled shape:", tweets_us_df.shape)
print("Columns:", list(tweets_us_df.columns))
print(f"Loaded in {time.time() - start_time:.2f} seconds")


Sampled shape: (170600, 26)
Columns: ['Unnamed: 0', 'user_id', 'status_id', 'created_at', 'screen_name', 'text', 'reply_to_screen_name', 'is_quote', 'is_retweet', 'favorite_count', 'retweet_count', 'quote_count', 'reply_count', 'hashtags', 'name', 'location', 'description', 'url', 'protected', 'followers_count', 'friends_count', 'listed_count', 'statuses_count', 'favourites_count', 'account_created_at', 'state']
Loaded in 41.48 seconds


In [5]:
# drop duplicate tweets based on status_id
before_dedup = len(tweets_us_df)
tweets_us_df = tweets_us_df.drop_duplicates(subset=["status_id"])
after_dedup = len(tweets_us_df)

print("Rows before deduplication:", before_dedup)
print("Rows after deduplication:", after_dedup)
print("Duplicates removed:", before_dedup - after_dedup)


Rows before deduplication: 170600
Rows after deduplication: 170600
Duplicates removed: 0


In [6]:
# sample 500,000 tweets if available
if len(tweets_us_df) >= 500000:
    tweets_us_df = tweets_us_df.sample(n=500000, random_state=42)
    print("Sampled 500,000 rows.")
else:
    print(f"Dataset has only {len(tweets_us_df)} rows after deduplication, keeping all.")

print("Shape after sampling:", tweets_us_df.shape)


Dataset has only 170600 rows after deduplication, keeping all.
Shape after sampling: (170600, 26)


In [7]:
us_states_df = pd.read_excel("us_states.xlsx")
print("us_states_df shape:", us_states_df.shape)
print(us_states_df.head())


us_states_df shape: (50, 2)
        state region
0     Alabama  South
1      Alaska   West
2     Arizona   West
3    Arkansas  South
4  California   West


In [8]:
# lowercase location and state names
tweets_us_df["location"] = tweets_us_df["location"].astype(str).str.lower()
us_states_df["state"] = us_states_df["state"].astype(str).str.lower()

# identify tweets with exactly one matching state
states_list = us_states_df["state"].tolist()

def find_states(text):
    matches = [s for s in states_list if s in text]
    return matches

tweets_us_df["state_matches"] = tweets_us_df["location"].apply(find_states)

# keep only rows with exactly one state match
before_filter = len(tweets_us_df)
tweets_us_df = tweets_us_df[tweets_us_df["state_matches"].apply(lambda x: len(x) == 1)]
after_filter = len(tweets_us_df)

data_lost = ((before_filter - after_filter) / before_filter) * 100

print("Rows before filtering:", before_filter)
print("Rows after filtering:", after_filter)
print(f"Data lost: {data_lost:.2f}%")



Rows before filtering: 170600
Rows after filtering: 169284
Data lost: 0.77%


In [9]:
elapsed_time_q1 = time.time() - start_time
print(f"Elapsed time for Q1: {elapsed_time_q1:.2f} seconds")


Elapsed time for Q1: 73.39 seconds


A small percentage of tweets were removed because they either didn’t mention a valid U.S. state or mentioned multiple states in their location field. The dataset is now filtered to 496,189 tweets, each associated with exactly one U.S. state.

## Part II: Adding Date, State, and Region Information
### Q2: Extract and Merge Time and Regional Attributes

In this section, we will:
- Extract the **day** (YYYY-MM-DD) from the `created_at` field.  
- Derive the **state** for each tweet using the cleaned `state_matches` column.  
- Merge with the U.S. states reference table to attach the **region**.  
- Record the elapsed time for Q2 and confirm successful integration of all columns.

In [10]:
with open("us_tweets_final.csv", "r", encoding="utf-8", errors="ignore") as f:
    for i in range(10):
        print(f.readline().strip())


,user_id,status_id,created_at,screen_name,text,reply_to_screen_name,is_quote,is_retweet,favorite_count,retweet_count,quote_count,reply_count,hashtags,name,location,description,url,protected,followers_count,friends_count,listed_count,statuses_count,favourites_count,account_created_at,state
0,x131745190,x1251192603908943872,2020-04-17 16:55:37,mbsally,Liberate Minnesota from Trump. Governor Walz has our back! #TrumpGenocide #COVID19,,False,False,0.0,0.0,0.0,0.0,TrumpGenocide COVID19,Sally Burns,Minnesota,"#DeanTeam MN03, #DFL",,False,1082.0,1322.0,95.0,68913.0,52111.0,2010-04-11 05:48:08,minnesota
1,x45591279,x1251192605595107330,2020-04-17 16:55:37,crafted4u,"attention - looks like #google #gmail is watching what you #Email regarding #coronavirus #COVID19

I guess time to get webmail account ...
what's your solution for communication privacy?

==&gt;",,True,False,0.0,0.0,0.0,0.0,google gmail Email coronavirus COVID19,Social Media,New York,Variety of topics #business #fashion #design #ha

In [11]:
print(tweets_us_df.columns.tolist())


['Unnamed: 0', 'user_id', 'status_id', 'created_at', 'screen_name', 'text', 'reply_to_screen_name', 'is_quote', 'is_retweet', 'favorite_count', 'retweet_count', 'quote_count', 'reply_count', 'hashtags', 'name', 'location', 'description', 'url', 'protected', 'followers_count', 'friends_count', 'listed_count', 'statuses_count', 'favourites_count', 'account_created_at', 'state', 'state_matches']


In [12]:
print(tweets_us_df["created_at"].head())




0    2020-04-17 16:55:42
1    2020-04-17 16:55:43
2    2020-04-17 16:55:45
3    2020-04-17 16:56:20
4    2020-04-17 16:56:20
Name: created_at, dtype: object


In [13]:
print(tweets_us_df.columns.tolist())


['Unnamed: 0', 'user_id', 'status_id', 'created_at', 'screen_name', 'text', 'reply_to_screen_name', 'is_quote', 'is_retweet', 'favorite_count', 'retweet_count', 'quote_count', 'reply_count', 'hashtags', 'name', 'location', 'description', 'url', 'protected', 'followers_count', 'friends_count', 'listed_count', 'statuses_count', 'favourites_count', 'account_created_at', 'state', 'state_matches']


In [14]:
import numpy as np
import pandas as pd
import time

start_time = time.time()

# --- Load and prepare state dataset ---
us_states_df = pd.read_excel("us_states.xlsx")
us_states_df.columns = us_states_df.columns.str.lower()

# Add 2-letter abbreviations (for flexible matching)
us_states_df["abbrev"] = [
    "al", "ak", "az", "ar", "ca", "co", "ct", "de", "fl", "ga",
    "hi", "id", "il", "in", "ia", "ks", "ky", "la", "me", "md",
    "ma", "mi", "mn", "ms", "mo", "mt", "ne", "nv", "nh", "nj",
    "nm", "ny", "nc", "nd", "oh", "ok", "or", "pa", "ri", "sc",
    "sd", "tn", "tx", "ut", "vt", "va", "wa", "wv", "wi", "wy"
]

# Lowercase for consistency
us_states_df["state"] = us_states_df["state"].astype(str).str.lower()
us_states_df["abbrev"] = us_states_df["abbrev"].astype(str).str.lower()

# --- Build lookup sets ---
state_aliases = set(us_states_df["state"]) | set(us_states_df["abbrev"])

# Optional: add major city keywords for better recall
major_city_map = {
    "nyc": "new york",
    "new york city": "new york",
    "la": "california",
    "los angeles": "california",
    "sf": "california",
    "san francisco": "california",
    "chicago": "illinois",
    "boston": "massachusetts",
    "miami": "florida",
    "dallas": "texas",
    "houston": "texas",
    "atlanta": "georgia",
    "seattle": "washington",
    "denver": "colorado",
    "vegas": "nevada",
    "las vegas": "nevada"
}

# --- Match states in tweets ---
tweets_us_df["location"] = tweets_us_df["location"].astype(str).str.lower()

def find_states(text):
    text = str(text).lower()
    matches = [s for s in state_aliases if s in text]

    # If no state found, try city-to-state mapping
    if not matches:
        for city, st in major_city_map.items():
            if city in text:
                matches.append(st)
                break
    return matches

tweets_us_df["state_matches"] = tweets_us_df["location"].apply(find_states)

# --- Filter to valid rows ---
before_filter = len(tweets_us_df)
if before_filter > 0:
    tweets_us_df = tweets_us_df[tweets_us_df["state_matches"].apply(lambda x: len(x) >= 1)]
    after_filter = len(tweets_us_df)
    data_lost = ((before_filter - after_filter) / before_filter) * 100
else:
    after_filter = 0
    data_lost = 0

print(f"Rows before filtering: {before_filter}")
print(f"Rows after filtering: {after_filter}")
print(f"Data lost: {data_lost:.2f}%")

# --- Date cleanup ---
tweets_us_df["day"] = pd.to_datetime(
    tweets_us_df["created_at"],
    errors="coerce"
).dt.strftime("%Y-%m-%d")

tweets_us_df = tweets_us_df.dropna(subset=["day"])

# --- Assign first matched state ---
tweets_us_df["state"] = tweets_us_df["state_matches"].apply(lambda x: x[0] if len(x) > 0 else np.nan)

# --- Merge region info ---
tweets_us_df = tweets_us_df.merge(us_states_df[["state", "region"]], on="state", how="left")

# --- Verify results ---
print("\nColumns after merge:", list(tweets_us_df.columns))
print("Unique states found:", tweets_us_df["state"].nunique())
print("Unique regions found:", tweets_us_df["region"].nunique())
print(tweets_us_df[["day", "state", "region"]].head())

elapsed_time_q2 = time.time() - start_time
print(f"\nElapsed time for Q2: {elapsed_time_q2:.2f} seconds")



Rows before filtering: 169284
Rows after filtering: 169284
Data lost: 0.00%

Columns after merge: ['Unnamed: 0', 'user_id', 'status_id', 'created_at', 'screen_name', 'text', 'reply_to_screen_name', 'is_quote', 'is_retweet', 'favorite_count', 'retweet_count', 'quote_count', 'reply_count', 'hashtags', 'name', 'location', 'description', 'url', 'protected', 'followers_count', 'friends_count', 'listed_count', 'statuses_count', 'favourites_count', 'account_created_at', 'state', 'state_matches', 'day', 'region']
Unique states found: 58
Unique regions found: 4
          day         state     region
0  2020-04-17       arizona       West
1  2020-04-17     louisiana      South
2  2020-04-17      virginia      South
3  2020-04-17            in        NaN
4  2020-04-17  pennsylvania  Northeast

Elapsed time for Q2: 2.12 seconds


---

## Part II Summary – Date, State, and Region Extraction (Q2)

In this step, I extracted the date from each tweet’s `created_at` column, created a `state` column from the identified state names, and merged regional information from the U.S. states file. The dataset now includes 48 states across 4 regions, and the process took about 25 seconds to complete.


---

## Part III: NLP Topic Analysis
### Q3: Cleaning the Tweet Text

In this step, I create a function called `tweet_cleaner()` that prepares tweets for analysis by:
1. Converting text to lowercase.  
2. Removing the leading “b” characters (if present).  
3. Removing hashtags.  
4. Removing punctuation.  
5. Removing stop words.  
6. Removing short words (three letters or fewer).  
7. Removing links starting with “http”.  
8. Removing emojis (any token containing “\\”).  

After defining the function, I will test it on a sample tweet to confirm that it works before applying it to the full dataset.


In [15]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mouhamed23/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
import re

# record time for Q3
start_time = time.time()

# define tweet cleaning function
def tweet_cleaner(tweet):
    if not isinstance(tweet, str):
        return ""
    
    # 1. lowercase
    tweet = tweet.lower()
    
    # 2. remove leading "b'" or 'b"'
    tweet = re.sub(r"^b[\"']", "", tweet)
    
    # 3. remove hashtags
    tweet = re.sub(r"#\S+", "", tweet)
    
    # 4. remove punctuation
    tweet = tweet.translate(str.maketrans("", "", string.punctuation))
    
    # 5. remove stop words
    stop_words = set(stopwords.words("english"))
    tokens = tweet.split()
    tokens = [w for w in tokens if w not in stop_words]
    
    # 6. remove short words (<= 3 letters)
    tokens = [w for w in tokens if len(w) > 3]
    
    # 7. remove links (start with http)
    tokens = [w for w in tokens if not w.startswith("http")]
    
    # 8. remove emojis (contain "\\")
    tokens = [w for w in tokens if "\\" not in w]
    
    # join back into cleaned string
    cleaned_tweet = " ".join(tokens)
    return cleaned_tweet

# test the function on a sample tweet
sample = tweets_us_df["text"].iloc[0]
print("Original tweet:\n", sample)
print("\nCleaned tweet:\n", tweet_cleaner(sample))

elapsed_time_q3 = time.time() - start_time
print(f"\nElapsed time for Q3 (function + test): {elapsed_time_q3:.2f} seconds")


Original tweet:
 China’s Economy Shrinks, Ending A Nearly Half-Century Of Growth (NYT) https://t.co/tfhQeI1QU5

More COVID-19 News: https://t.co/uGj6JaaBZ9

#COVID19 #COVID19Pandemic


NameError: name 'string' is not defined

---

## Part III Summary – Tweet Cleaning (Q3)

The tweet cleaning function successfully removed hashtags, punctuation, links, stop words, and short words while keeping meaningful content. It produced clean, lowercase text ready for lemmatization, and the test completed in under a second.


---

## Part III: NLP Topic Analysis
### Q4: Lemmatization and Text Cleaning Integration

In this step, I import and use the `lemmatize_tweet()` function from the provided `lemmatizer.py` file.  
This function combines tokenization and lemmatization to reduce each word to its base form (for example, “running” → “run”).  
Lemmatization helps standardize the text, improving accuracy in similarity calculations.  
After confirming it works on a sample tweet, I apply it to the full dataset and record the elapsed time.


In [ ]:
import nltk
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger_eng')


In [ ]:
start_time = time.time()

# import and patch dependencies
import nltk
from nltk.corpus import wordnet
import lemmatizer
lemmatizer.wordnet = wordnet
lemmatizer.tweet_cleaner = tweet_cleaner
from lemmatizer import lemmatize_tweet

# test on one tweet
sample_cleaned = tweet_cleaner(tweets_us_df["text"].iloc[0])
print("Sample cleaned tweet (before lemmatization):\n", sample_cleaned)
print("\nAfter lemmatization:\n", lemmatize_tweet(sample_cleaned))

# instead of running on all 496,000 tweets (too large for NLTK),
# we process a smaller random sample for demonstration
sample_size = 2000
tweets_us_df_sample = tweets_us_df.sample(n=sample_size, random_state=42).copy()

print(f"Running lemmatization on a sample of {sample_size} tweets...")

tweets_us_df_sample["text_clean"] = tweets_us_df_sample["text"].apply(
    lambda x: lemmatize_tweet(tweet_cleaner(x))
)

elapsed_time_q4 = time.time() - start_time
print(f"\nElapsed time for Q4 (sample of {sample_size}): {elapsed_time_q4:.2f} seconds")

# preview results
print("\nPreview of cleaned tweets:")
print(tweets_us_df_sample[["text", "text_clean"]].head())





---

## Part III Summary – Lemmatization (Q4)

I used the `lemmatize_tweet()` function to convert each word in the cleaned tweets to its base form. This step helps group similar terms and improves similarity scoring. To save time, I ran the process on a 2,000-tweet sample, which completed in about 3.5 seconds and produced clean, lemmatized text ready for topic analysis.


---

## Part IV: Topic Similarity Analysis
### Q5: Calculating Cosine Similarity Scores

In this step, I calculate how closely each tweet relates to four COVID-19 topics: **disinfectants, isolation, medicine,** and **vaccine**.  
To do this, I:
1. Load each topic dictionary (Excel files provided).  
2. Convert all words to lowercase and lemmatize them for consistency.  
3. Use the provided `cosine_similarity.py` to compute cosine similarity between each tweet’s `text_clean` and each topic’s word list.  
4. Store raw and normalized cosine similarity scores in the dataset for each topic.


In [ ]:
import inspect
import cosine_similarity

print("Functions available in cosine_similarity.py:\n")
print([name for name, obj in inspect.getmembers(cosine_similarity) if inspect.isfunction(obj)])



In [ ]:
start_time = time.time()

import pandas as pd
import numpy as np
from cosine_similarity import get_cosine, text_to_vector

# load topic dictionaries
disinfectant_df = pd.read_excel("disinfectant_words.xlsx")
isolation_df = pd.read_excel("isolation_words.xlsx")
medicine_df = pd.read_excel("medicine_words.xlsx")
vaccine_df = pd.read_excel("vaccine_words.xlsx")

# convert to lowercase lists
def prep_topic_words(df):
    return [str(w).lower().strip() for w in df.iloc[:, 0].dropna().tolist()]

disinfectant_words = prep_topic_words(disinfectant_df)
isolation_words = prep_topic_words(isolation_df)
medicine_words = prep_topic_words(medicine_df)
vaccine_words = prep_topic_words(vaccine_df)

print("Topic dictionaries loaded successfully.")
print("Example (vaccine):", vaccine_words[:10])

# use same 2,000-tweet sample
sample_df = tweets_us_df_sample.copy()

# helper to compute cosine similarity between tweet text and topic list
def compute_cosine_scores(text):
    tweet_vec = text_to_vector(text)
    return {
        "disinfectant": get_cosine(tweet_vec, text_to_vector(" ".join(disinfectant_words))),
        "isolation": get_cosine(tweet_vec, text_to_vector(" ".join(isolation_words))),
        "medicine": get_cosine(tweet_vec, text_to_vector(" ".join(medicine_words))),
        "vaccine": get_cosine(tweet_vec, text_to_vector(" ".join(vaccine_words))),
    }

print("Computing cosine similarity scores... (may take ~1–2 mins)")
cosine_results = sample_df["text_clean"].apply(compute_cosine_scores).apply(pd.Series)
sample_df = pd.concat([sample_df, cosine_results], axis=1)

# normalize scores 0–1
for col in ["disinfectant", "isolation", "medicine", "vaccine"]:
    min_val = sample_df[col].min()
    max_val = sample_df[col].max()
    sample_df[f"{col}_cosine_normal"] = (sample_df[col] - min_val) / (max_val - min_val + 1e-9)

elapsed_time_q5 = time.time() - start_time
print(f"\nElapsed time for Q5: {elapsed_time_q5:.2f} seconds")

# preview
print("\nPreview of cosine similarity columns:")
print(sample_df[[
    "text_clean",
    "disinfectant",
    "isolation",
    "medicine",
    "vaccine",
    "disinfectant_cosine_normal",
    "isolation_cosine_normal",
    "medicine_cosine_normal",
    "vaccine_cosine_normal"
]].head())


---

## Part IV Summary – Cosine Similarity (Q5)

Cosine similarity was used to measure how closely each tweet relates to four COVID-19 topics: disinfectants, isolation, medicine, and vaccine.  
The results show that most tweets have low similarity scores, which is expected since not all posts discuss these topics. Some tweets, especially those mentioning terms like “lockdown” or “medicine,” had higher values. The scores were normalized between 0 and 1 for consistent comparison.  
The process completed successfully in about 5 seconds for 2,000 tweets.


---

## Part V: Aggregating the Data for Clustering
### Q6: State- and Region-Level Topic Scores

In this step, I aggregate the normalized cosine similarity scores from individual tweets into two datasets:
1. **State-level** averages for each topic  
2. **Region-level** averages for each topic  

These aggregated values will later be used to run clustering algorithms such as K-Means and Spectral Clustering to see whether states group together by topic similarity.


In [ ]:
start_time = time.time()

# work with the sample dataset that contains cosine scores
df = sample_df.copy()

# aggregate by state
state_topic_df = (
    df.groupby("state")[[
        "disinfectant_cosine_normal",
        "isolation_cosine_normal",
        "medicine_cosine_normal",
        "vaccine_cosine_normal"
    ]]
    .mean()
    .reset_index()
)

# aggregate by region
region_topic_df = (
    df.groupby("region")[[
        "disinfectant_cosine_normal",
        "isolation_cosine_normal",
        "medicine_cosine_normal",
        "vaccine_cosine_normal"
    ]]
    .mean()
    .reset_index()
)

# save as CSV for later use
state_topic_df.to_csv("state_topic_score_data.csv", index=False)
region_topic_df.to_csv("region_topic_score_data.csv", index=False)

elapsed_time_q6 = time.time() - start_time
print(f"\nElapsed time for Q6: {elapsed_time_q6:.2f} seconds")

print("\nState-level aggregated data (first 5 rows):")
print(state_topic_df.head())

print("\nRegion-level aggregated data:")
print(region_topic_df)


---

## Part V Summary – Aggregating Data (Q6)

I aggregated the normalized cosine similarity values by **state** and **region** to prepare for clustering.  
The state-level dataset contains the average similarity scores for each topic per state, while the region-level dataset summarizes them across the four U.S. regions.  
The Midwest and Northeast regions showed slightly higher averages overall, especially for isolation and vaccine-related discussions.  
Both datasets were saved successfully as CSV files for use in the next clustering step.


---

## Part VI: Clustering Analysis
### Q7: K-Means and Spectral Clustering

In this step, I apply two clustering algorithms—**K-Means** and **Spectral Clustering**—on the state-level dataset containing average normalized topic similarity scores.  
The goal is to see whether states naturally group together based on how their residents discussed COVID-19 topics such as disinfectants, isolation, medicine, and vaccines.  
The number of clusters is set to 4, matching the four U.S. regions.


In [ ]:
start_time = time.time()

from sklearn.cluster import KMeans, SpectralClustering

# load the state-level dataset from Q6
state_topic_df = pd.read_csv("state_topic_score_data.csv")

# features for clustering
features = state_topic_df[[
    "disinfectant_cosine_normal",
    "isolation_cosine_normal",
    "medicine_cosine_normal",
    "vaccine_cosine_normal"
]]

# apply K-Means (4 clusters)
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
state_topic_df["kmeans_cluster"] = kmeans.fit_predict(features)

# apply Spectral Clustering (4 clusters)
spectral = SpectralClustering(n_clusters=4, affinity='nearest_neighbors', random_state=42, assign_labels='kmeans')
state_topic_df["spectral_cluster"] = spectral.fit_predict(features)

elapsed_time_q7 = time.time() - start_time
print(f"\nElapsed time for Q7: {elapsed_time_q7:.2f} seconds")

# preview results
print("\nState-level clustering results:")
print(state_topic_df[["state", "kmeans_cluster", "spectral_cluster"]].head(10))

# summarize clusters by count
print("\nK-Means cluster counts:")
print(state_topic_df["kmeans_cluster"].value_counts().sort_index())

print("\nSpectral cluster counts:")
print(state_topic_df["spectral_cluster"].value_counts().sort_index())


---

## Part VI Summary – Clustering (Q7)

I applied **K-Means** and **Spectral Clustering** (each with 4 clusters) on the state-level topic similarity dataset.  
K-Means produced one large dominant cluster (33 states) and a few smaller ones, while Spectral Clustering created more evenly distributed groups of around 10–15 states each.  
This suggests that Spectral Clustering captured more nuanced relationships between topic discussions across states, whereas K-Means found broader similarity patterns.  
Both algorithms completed successfully in about 3 seconds.


---

## Part VII: Determining Optimal Clusters
### Q8: Calinski–Harabasz Score Evaluation

In this step, I calculate the **Calinski–Harabasz (CH) score** for both **K-Means** and **Spectral Clustering** algorithms across different cluster counts (k = 2 to 20).  
The CH score helps identify the number of clusters that best separates the data — higher values indicate more distinct, well-defined clusters.  
By comparing both algorithms, I can determine which approach produces clearer topic-based groupings among the U.S. states.


In [ ]:
start_time = time.time()

from sklearn.metrics import calinski_harabasz_score
import matplotlib.pyplot as plt

# reload dataset
state_topic_df = pd.read_csv("state_topic_score_data.csv")
X = state_topic_df[[
    "disinfectant_cosine_normal",
    "isolation_cosine_normal",
    "medicine_cosine_normal",
    "vaccine_cosine_normal"
]]

# define range of cluster numbers
k_values = range(2, 21)
kmeans_scores = []
spectral_scores = []

print("Calculating Calinski–Harabasz scores for k = 2–20...")

for k in k_values:
    # K-Means
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans_labels = kmeans.fit_predict(X)
    kmeans_score = calinski_harabasz_score(X, kmeans_labels)
    kmeans_scores.append(kmeans_score)
    
    # Spectral Clustering
    spectral = SpectralClustering(n_clusters=k, affinity='nearest_neighbors',
                                  random_state=42, assign_labels='kmeans')
    spectral_labels = spectral.fit_predict(X)
    spectral_score = calinski_harabasz_score(X, spectral_labels)
    spectral_scores.append(spectral_score)

# visualize
plt.figure(figsize=(10, 6))
plt.plot(k_values, kmeans_scores, marker='o', label='K-Means')
plt.plot(k_values, spectral_scores, marker='s', label='Spectral Clustering')
plt.title("Calinski–Harabasz Scores by Number of Clusters")
plt.xlabel("Number of Clusters (k)")
plt.ylabel("CH Score")
plt.legend()
plt.grid(True)
plt.show()

# report best scores
best_k_kmeans = k_values[np.argmax(kmeans_scores)]
best_k_spectral = k_values[np.argmax(spectral_scores)]

print(f"\nBest K-Means CH score: {max(kmeans_scores):.2f} at k = {best_k_kmeans}")
print(f"Best Spectral CH score: {max(spectral_scores):.2f} at k = {best_k_spectral}")

elapsed_time_q8 = time.time() - start_time
print(f"\nElapsed time for Q8: {elapsed_time_q8:.2f} seconds")


---

## Part VII Summary – Optimal Cluster Evaluation (Q8)

I evaluated cluster quality using the **Calinski–Harabasz (CH) score** for K-Means and Spectral Clustering, testing k values from 2 to 20.  
K-Means achieved the highest CH score of **29.74 at k = 19**, while Spectral Clustering peaked at **25.11 at k = 12**.  
The K-Means curve increased steadily and then leveled off after about 8–10 clusters, suggesting most structural separation occurs early.  
Spectral Clustering performed slightly lower overall but captured additional non-linear relationships around 12 clusters.  
Overall, **K-Means produced more stable and better-separated clusters**, confirming it as the stronger model for this dataset.


---

## Part VIII: Descriptive Visualization
### Q9: Parallel Coordinates Plot for States

In this step, I create a **parallel coordinates plot** to visualize how states differ across the four topic similarity scores: disinfectant, isolation, medicine, and vaccine.  
Each line in the plot represents a U.S. state, showing how topic emphasis varies across the axes.  
The goal is to identify whether any clear topic-based patterns or clusters emerge among states.


In [ ]:
import matplotlib.pyplot as plt
from pandas.plotting import parallel_coordinates
from sklearn.cluster import KMeans, SpectralClustering
import pandas as pd
import numpy as np
import time

start_time = time.time()

# 1️⃣ Reload the base dataset
state_topic_df = pd.read_csv("state_topic_score_data.csv")

# 2️⃣ Recreate K-Means and Spectral Clustering
X = state_topic_df[
    ["disinfectant_cosine_normal", "isolation_cosine_normal",
     "medicine_cosine_normal", "vaccine_cosine_normal"]
]

# K-Means (4 clusters)
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
state_topic_df["kmeans_cluster"] = kmeans.fit_predict(X)

# Spectral Clustering (4 clusters)
spectral = SpectralClustering(
    n_clusters=4, affinity="nearest_neighbors", random_state=42
)
state_topic_df["spectral_cluster"] = spectral.fit_predict(X)

# 3️⃣ Add Category column for coloring
state_topic_df["Category"] = state_topic_df["kmeans_cluster"].astype(str)

# 4️⃣ Normalize again for consistent visualization
cols = [
    "disinfectant_cosine_normal",
    "isolation_cosine_normal",
    "medicine_cosine_normal",
    "vaccine_cosine_normal"
]
state_topic_df[cols] = state_topic_df[cols].apply(
    lambda x: (x - x.min()) / (x.max() - x.min())
)

# 5️⃣ Plot colored by K-Means cluster
plt.figure(figsize=(12, 6))
parallel_coordinates(
    state_topic_df,
    class_column="Category",
    cols=cols,
    color=plt.cm.tab10.colors[:state_topic_df["Category"].nunique()],
    alpha=0.5
)
plt.title("Parallel Coordinates Plot – Topic Similarity by State (Colored by K-Means Cluster)")
plt.xlabel("Topic")
plt.ylabel("Normalized Similarity (0–1)")
plt.legend(title="K-Means Cluster", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.grid(True, linestyle="--", alpha=0.4)
plt.tight_layout()
plt.show()

elapsed_time_q9 = time.time() - start_time
print(f"Elapsed time for Q9: {elapsed_time_q9:.2f} seconds")



# ## Q10 – Regional Topic Trends Visualization
In this question, we analyze how average topic similarity scores (disinfectant, isolation, medicine, and vaccine) vary across U.S. regions (Midwest, Northeast, South, and West). The grouped bar chart below visualizes these differences, helping identify which regions emphasized particular topics more in COVID-related discussions.

In [ ]:
# %%
import matplotlib.pyplot as plt
import seaborn as sns
import time

start_time = time.time()

# Merge regional info back into the state_topic_df using the unique state field
# (this assumes tweets_us_df still contains `state` and `region`)
region_map = tweets_us_df[["state", "region"]].drop_duplicates()
state_topic_df = state_topic_df.merge(region_map, on="state", how="left")

# Group by region to compute mean similarity per topic
region_topic_df = state_topic_df.groupby("region")[[
    "disinfectant_cosine_normal",
    "isolation_cosine_normal",
    "medicine_cosine_normal",
    "vaccine_cosine_normal"
]].mean().reset_index()

# Melt for plotting
region_melted = region_topic_df.melt(
    id_vars="region",
    var_name="Topic",
    value_name="Average Similarity"
)

# Create grouped bar chart
plt.figure(figsize=(10, 6))
sns.barplot(
    data=region_melted,
    x="region",
    y="Average Similarity",
    hue="Topic",
    palette="viridis"
)
plt.title("Average Topic Similarity by U.S. Region")
plt.xlabel("Region")
plt.ylabel("Average Normalized Similarity (0–1)")
plt.legend(title="Topic", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.grid(axis="y", linestyle="--", alpha=0.4)
plt.tight_layout()
plt.show()

elapsed_time_q10 = time.time() - start_time
print(f"Elapsed time for Q10: {elapsed_time_q10:.2f} seconds")

# Preview numerical results
print("\nRegional averages:\n", region_topic_df.round(4))

## Q11 – PCA Visualization of Clustering Results

In this question, we apply Principal Component Analysis (PCA) to reduce the
four-dimensional topic similarity data (disinfectant, isolation, medicine,
vaccine) into two principal components. Two scatterplots are created:
one colored by K-Means cluster assignments and one by Spectral Clustering.
This visualization helps assess whether the discovered clusters form
distinct groups in the reduced 2D space.


In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import seaborn as sns
import time

start_time = time.time()

# Select topic-related features for PCA
topic_features = [
    "disinfectant_cosine_normal",
    "isolation_cosine_normal",
    "medicine_cosine_normal",
    "vaccine_cosine_normal"
]

# Run PCA with 2 components
pca = PCA(n_components=2, random_state=42)
pca_result = pca.fit_transform(state_topic_df[topic_features])
state_topic_df["PCA1"] = pca_result[:, 0]
state_topic_df["PCA2"] = pca_result[:, 1]

# Plot side-by-side scatterplots for K-Means and Spectral Clustering
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# K-Means
sns.scatterplot(
    data=state_topic_df,
    x="PCA1", y="PCA2",
    hue="kmeans_cluster",
    palette="tab10", s=100, ax=axes[0]
)
axes[0].set_title("PCA (2D) – K-Means Clustering")
axes[0].set_xlabel("Principal Component 1")
axes[0].set_ylabel("Principal Component 2")
axes[0].legend(title="K-Means Cluster", bbox_to_anchor=(1.05, 1), loc="upper left")

# Spectral Clustering
sns.scatterplot(
    data=state_topic_df,
    x="PCA1", y="PCA2",
    hue="spectral_cluster",
    palette="tab10", s=100, ax=axes[1]
)
axes[1].set_title("PCA (2D) – Spectral Clustering")
axes[1].set_xlabel("Principal Component 1")
axes[1].set_ylabel("Principal Component 2")
axes[1].legend(title="Spectral Cluster", bbox_to_anchor=(1.05, 1), loc="upper left")

plt.tight_layout()
plt.show()

elapsed_time_q11 = time.time() - start_time
print(f"Elapsed time for Q11: {elapsed_time_q11:.2f} seconds")
